# Programming Assignment 4 - Linear versus Ridge Regression 

The Boston housing data set was collected in the 1970s to study the relationship between house price and various factors such as the house size, crime rate, socio-economic status, etc. 

 



In this notebook, you are to explore the effects of ridge regression.  We will use a dataset that is part of the sklearn.dataset package.  Learn more at https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

## Step 1:  Getting, understanding, and preprocessing the dataset

We first import the standard libaries and some libraries that will help us scale the data and perform some "feature engineering" by transforming the data into $\Phi_2({\bf x})$

In [2]:
import numpy as np
import sklearn
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.linear_model
from sklearn.model_selection import KFold

###  Importing the dataset

In [27]:
# Import the boston dataset from sklearn
boston_data = load_boston()

In [28]:
#  Create X and Y variables - X holding the design matrix and Y holding target vector
X_orig = boston_data.data 
y = boston_data.target

# Proprocesing by adding a column of 1's to the front of X_orig to create X
one_col = np.ones((X_orig.shape[0],1))
X = np.hstack((one_col, X_orig))

#  Reshape Y to be a rank 2 matrix 
y = y.reshape(X_orig.shape[0], 1)

# Observe the number of features and the number of labels
print('The number of features +1 (for the intercept) is: ', X.shape[1])
# Printing out the features
print('The features: ', boston_data.feature_names)
# The number of examples
print('The number of exampels in our dataset: ', X.shape[0])
#Observing the first 2 rows of the data
print(X[0:2])


The number of features +1 (for the intercept) is:  14
The features:  ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
The number of exampels in our dataset:  506
[[1.0000e+00 6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01
  6.5750e+00 6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01
  3.9690e+02 4.9800e+00]
 [1.0000e+00 2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01
  6.4210e+00 7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01
  3.9690e+02 9.1400e+00]]


Since we will test linear and ridge regression on data with degree = 1 and data with degree = 2, we next create polynomial features of  degree 2 using the original dataset.  Feel free to increase the # of degress and see what effect it has on the training and test error. 

In [5]:
# Create a PolynomialFeatures object with degree = 2. 
# Transform X and save it into X_2. Simply copy Y into Y_2 
# Note: PolynomialFeatures creates a column of ones as the first feature
poly = PolynomialFeatures(degree=2,include_bias = True) 
X_2 = poly.fit_transform(X_orig)
y_2 = y


In [6]:
# the shape of X_2 and Y_2 - should be (506, 105) and (506, 1) respectively
print(X_2.shape)
print(y_2.shape)

(506, 105)
(506, 1)


# Your code goes here

In [7]:
# TODO - Define the get_coeff_ridge_normaleq function. Use the normal equation method.
# TODO - Return w values

def get_coeff_ridge_normaleq(X_train, y_train, alpha):
    # use np.linalg.pinv(a)
    #### TO-DO #####
    term1 = np.dot(np.transpose(X_train), X_train)
    identity = np.identity(len(term1))
    identity[0][0] = 0
    term2 = len(X_train)*alpha*identity
    term_sum = term1 + term2
    inv = np.linalg.pinv(term_sum)
    product = np.dot(inv, np.transpose(X_train))
    w = np.dot(product, y_train)
    ##############
    return w

In [8]:
# TODO - Define the evaluate_err_ridge function.
# TODO - Return the train_error and test_error values

def evaluate_err(X_train, X_test, y_train, y_test, w): 
    #### TO-DO #####
    train_error = 0
    test_error = 0
    train = np.dot(X_train, w)
    test = np.dot(X_test, w)
    for i in range(len(y_train)):
        diff = y_train[i] - train[i]
        sq = np.square(diff)
        train_error += sq
    for j in range(len(y_test)):
        diff = y_test[j] - test[j]
        sq = np.square(diff)
        test_error += sq
    train_error = train_error/len(X_train)
    test_error = test_error/len(X_test)
    ##############
    return train_error, test_error

In [9]:
# TODO - Finish writting the k_fold_cross_validation function. 
# TODO - Returns the average training error and average test error from the k-fold cross validation
# use Sklearns K-Folds cross-validator: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

def k_fold_cross_validation(k, X, y, alpha):
    kf = KFold(n_splits=k, random_state=10, shuffle=True)
    total_E_val_test = 0
    total_E_val_train = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # scaling the data matrix (except for for the first column of ones)
        scaler = preprocessing.StandardScaler().fit(X_train[:,1:(X_train.shape[1]+1)])
        X_train[:,1:(X_train.shape[1]+1)] = scaler.transform(X_train[:,1:(X_train.shape[1]+1)])
        X_test[:,1:(X_train.shape[1]+1)] = scaler.transform(X_test[:,1:(X_train.shape[1]+1)])
        
        # determine the training error and the test error
        #### TO-DO #####
        w_values = get_coeff_ridge_normaleq(X_train, y_train, alpha)
        err = evaluate_err(X_train, X_test, y_train, y_test, w_values)
        total_E_val_train += err[0]
        total_E_val_test += err[1]
    E_val_train = total_E_val_train/k
    E_val_test = total_E_val_test/k
       ##############
    return  E_val_test, E_val_train

In [10]:
# linear regression MSE, set alpha to 0
average_MSE = k_fold_cross_validation(3, X, y, 0)
print("The average MSE using linear regression for {0}-fold cross validation for the training set is: {1}".format(3, average_MSE[1]))
print("The average MSE using linear regression for {0}-fold cross validation for the test set is: {1}".format(3, average_MSE[0]))

The average MSE using linear regression for 3-fold cross validation for the training set is: [21.4928993]
The average MSE using linear regression for 3-fold cross validation for the test set is: [24.00263612]


In [18]:
# ridge regression MSE
alpha = np.logspace(-5, 1, num=15)
results = []
for i in alpha:
    output = k_fold_cross_validation(3, X, y, i)
    results.append(output)
print(results)

[(array([24.00251564]), array([21.49289933])), (array([24.00231308]), array([21.49289952])), (array([24.00177086]), array([21.49290083])), (array([24.00032462]), array([21.49291023])), (array([23.99650434]), array([21.49297731])), (array([23.98667234]), array([21.49344923])), (array([23.96307955]), array([21.49665058])), (array([23.9163919]), array([21.51653765])), (array([23.86850016]), array([21.61943029])), (array([23.97640239]), array([22.02158791])), (array([24.7253926]), array([23.23248286])), (array([27.42508676]), array([26.42508613])), (array([34.30346759]), array([33.57131725])), (array([45.8540057]), array([45.19188867])), (array([59.77419252]), array([59.14840907]))]


In [135]:
# experimenting with our model
# add m-smoothing of 0.01 to the data
# fit model with different values of alpha
m_smooth_x = []
m_smooth_y = []
alpha2 = np.logspace(-3, 3, num=15)
results2 =[]
for sample in X:
    arr = []
    for feature in sample:
        if feature != 1:
            feature += 0.01
        arr.append(feature)
    m_smooth_x.append(arr)
for element in y:
    element[0] += 0.01
    m_smooth_y.append(element)
mx = np.asarray(m_smooth_x)
my = np.asarray(m_smooth_y)
for a in alpha2:
    output = k_fold_cross_validation(3, mx, my, a)
    results2.append(output)
print(alpha2)
print(results2)

[1.00000000e-03 2.68269580e-03 7.19685673e-03 1.93069773e-02
 5.17947468e-02 1.38949549e-01 3.72759372e-01 1.00000000e+00
 2.68269580e+00 7.19685673e+00 1.93069773e+01 5.17947468e+01
 1.38949549e+02 3.72759372e+02 1.00000000e+03]
[(array([23.99065661]), array([21.4927324])), (array([23.97280654]), array([21.49443456])), (array([23.93425497]), array([21.50541485])), (array([23.8789788]), array([21.56656897])), (array([23.90280735]), array([21.83062238])), (array([24.34911428]), array([22.68706967])), (array([26.17265508]), array([25.02261285])), (array([31.44374603]), array([30.65373143])), (array([41.58783503]), array([40.91131686])), (array([55.0973428]), array([54.4597634])), (array([68.32115404]), array([67.71583814])), (array([77.30379443]), array([76.71752356])), (array([81.7997244]), array([81.22189363])), (array([83.69756621]), array([83.12307846])), (array([84.4403495]), array([83.86713514]))]


In [136]:
# experiment with other values of alpha
alpha3 = np.logspace(-1, 5, num=15)
results3 = []
for a in alpha3:
    output = k_fold_cross_validation(3, mx, my, a)
    results3.append(output)
print(alpha3)
print(results3) 

[1.00000000e-01 2.68269580e-01 7.19685673e-01 1.93069773e+00
 5.17947468e+00 1.38949549e+01 3.72759372e+01 1.00000000e+02
 2.68269580e+02 7.19685673e+02 1.93069773e+03 5.17947468e+03
 1.38949549e+04 3.72759372e+04 1.00000000e+05]
[(array([24.11436488]), array([22.29678143])), (array([25.30532553]), array([23.98683072])), (array([29.15853664]), array([28.28003054])), (array([37.7077865]), array([37.01042629])), (array([50.40125619]), array([49.75157015])), (array([64.24137437]), array([63.62652662])), (array([74.87834538]), array([74.28721498])), (array([80.68610578]), array([80.10625298])), (array([83.2462438]), array([82.67097274])), (array([84.26663081]), array([83.69312038])), (array([84.65686469]), array([84.08401783])), (array([84.80373883]), array([84.23114032])), (array([84.85868563]), array([84.28617984])), (array([84.8791952]), array([84.30672399])), (array([84.88684418]), array([84.31438586]))]


In [145]:
# linear regression using degree 2 transformed features
lin_deg2 = k_fold_cross_validation(3, X_2, y_2, 0)
print(lin_deg2)

(array([14.59735947]), array([5.07272551]))


In [137]:
# ridge regression using degree 2 transformed features
results_deg2 = []
for i in alpha:
    output = k_fold_cross_validation(3, X_2, y_2, i)
    results_deg2.append(output)
print(alpha)
print(results_deg2)

[1.00000000e-05 2.68269580e-05 7.19685673e-05 1.93069773e-04
 5.17947468e-04 1.38949549e-03 3.72759372e-03 1.00000000e-02
 2.68269580e-02 7.19685673e-02 1.93069773e-01 5.17947468e-01
 1.38949549e+00 3.72759372e+00 1.00000000e+01]
[(array([13.67762186]), array([5.12547691])), (array([13.30411993]), array([5.19037096])), (array([13.03974159]), array([5.32082688])), (array([12.87863654]), array([5.55402986])), (array([12.75779823]), array([5.92693193])), (array([12.58948562]), array([6.46317457])), (array([12.45045465]), array([7.23599292])), (array([12.74780895]), array([8.35639849])), (array([14.04865221]), array([10.0120801])), (array([16.62208284]), array([12.44988098])), (array([19.56377178]), array([15.340242])), (array([21.94754404]), array([18.24288042])), (array([24.37926636]), array([21.68060466])), (array([29.02855827]), array([27.3047267])), (array([38.02851002]), array([36.8406942]))]


In [34]:
# scaling the data to make a prediction using ridge regression
test_sample = [[1, 0.1, 11, 7, 0, 0.4, 6, 70, 4, 6, 300, 16, 360, 10]]
scaler = preprocessing.StandardScaler().fit(X)
X_train_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(test_sample)
for i in X_train_scaled:
    i[0] = 1
X_test_scaled[0][0] = 1
w = get_coeff_ridge_normaleq(X_train_scaled, y, 0.026826957952797246)
prediction = X_test_scaled.dot(w)
print(w)
print(prediction)

[[22.53280632]
 [-0.83996967]
 [ 0.91985356]
 [-0.08594796]
 [ 0.71401554]
 [-1.740073  ]
 [ 2.75941742]
 [-0.04633409]
 [-2.77808299]
 [ 1.95211662]
 [-1.44106738]
 [-1.965708  ]
 [ 0.84285053]
 [-3.5855252 ]]
[[27.37819117]]
